In [ ]:
%pip install opencv-python numpy albumentationsx tensorflow keras keras.utils git+https://github.com/JanMarcelKezmann/TensorFlow-Advanced-Segmentation-Models.git tensorflow[and-cuda]

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard
import keras.backend as K
import albumentations as alb

from unet import create_unet_efficientnet

In [ ]:
# changeable values
DATASET_FOLDER = 'dataset_split_segmentation'

MODEL = 'efficientnetb3'
INPUT_SIZE = (512,512)
CLASSES = 4
CLASS_NAMES = ['background', 'glioma', 'meningioma', 'pituitary']

BATCH_SIZE = 8
EPOCHS = 100 # with EarlyStopping


In [ ]:
#load dataset into numpy array
import os
import numpy as np
import cv2

def load_data(image_folder_path: str, mask_folder_path: str,) -> np.ndarray:
    images = []
    masks = []
    tumor_types = ['glioma', 'meningioma', 'pituitary']
    for tumor_index, tumor_type in enumerate(tumor_types):
        img_folder = os.path.join(image_folder_path, tumor_type)
        mask_folder = os.path.join(mask_folder_path, tumor_type)

        for file in os.listdir(img_folder):
            image_path = os.path.join(img_folder, file)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,INPUT_SIZE, interpolation=cv2.INTER_LINEAR)
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
            image = image / 255.0

            mask_path = os.path.join(mask_folder, file)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, INPUT_SIZE, interpolation=cv2.INTER_NEAREST)
            mask = (mask > 127).astype(np.float32)
            mask = mask *(tumor_index + 1)

            images.append(image)
            masks.append(mask)
    
    images_arr = np.array(images)
    masks_arr = np.array(masks)

    masks_arr = np.expand_dims(masks_arr, axis=-1)

    return images_arr, masks_arr

test_images_arr, test_masks_arr = load_data('dataset_split_segmentation/test/images', 'dataset_split_segmentation/test/masks')
val_images_arr, val_masks_arr = load_data('dataset_split_segmentation/val/images', 'dataset_split_segmentation/val/masks')
train_images_arr, train_masks_arr = load_data('dataset_split_segmentation/train/images', 'dataset_split_segmentation/train/masks')


In [4]:
#setup augmentation
import albumentations as alb

training_augmentation = alb.Compose([
    alb.HorizontalFlip(p=0.5),
    alb.Rotate(limit=25, p=0.7),
    alb.Affine(scale=0.15, translate_percent=0.1, rotate=25, p=0.5),

    alb.ElasticTransform(alpha=0.5, sigma=30, p=0.2),
    alb.GridDistortion(num_steps=5, distort_limit=0.3, p=0.15),

    #image only
    alb.RandomBrightnessContrast(p=0.5),
    alb.RandomGamma(gamma_limit=(80, 120), p=0.5),

    alb.GaussianBlur(blur_limit=(3,7), p=0.5),
    alb.GaussNoise(std_range=(0.01, 0.03), p=0.2),

    alb.CLAHE(clip_limit=4.0, p=0.5),

    alb.CoarseDropout(num_holes_range=(1,4), hole_height_range=(8,16), hole_width_range =(8,16), fill=0, p=0.1)
])

validation_augmentation = alb.Compose([])

In [ ]:
import tensorflow as tf
from keras.utils import to_categorical

class SegmentationGenerator(tf.keras.utils.Sequence):
    def __init__(self,images_arr, masks_arr, BATCH_SIZE, CLASSES, augmentation = None, shuffle=True):
        self.images = images_arr
        self.masks = masks_arr
        self.batch_size = BATCH_SIZE
        self.classes = CLASSES
        self.augmentation = augmentation
        self.shuffle = shuffle
        self.indices = np.arange(len(images_arr))

        if shuffle:
            np.random.shuffle(self.indices)
    
    def __len__(self):
        return len(self.images) // self.batch_size
        
    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        batch_images = []
        batch_masks = []

        for i in batch_indices:
            img = self.images[i]
            mask = self.masks[i].squeeze()
        
            if self.augmentation:
                augmented = self.augmentation(image=img, mask=mask)
                img = augmented['image']
                mask = augmented['mask']

            batch_images.append(img)
            batch_masks.append(mask)

        batch_images = np.array(batch_images)
        batch_masks = np.array(batch_masks)

        batch_masks_onehot = to_categorical(
            batch_masks, 
            num_classes=self.classes
        )
        return batch_images, batch_masks_onehot

    def epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)


IndentationError: expected an indented block after function definition on line 17 (1710057207.py, line 18)

In [ ]:
from unet import create_unet_efficientnet

model = create_unet_efficientnet(
    shape_input=(512, 512, 3),
    classes=CLASSES,
    encoder_freeze=False,
    decoder_sizes=[512, 256, 128, 64],
    dropout_rate=0.3
)

In [ ]:
#loss funkce
import keras.backend as K

def dice_sum(y_true, y_pred, smooth = 1e-6):
    dice_sum = 0.0
    
    for class_idx in range(4):
        y_true_class = y_true[..., class_idx]
        y_pred_class = y_pred[..., class_idx]
        
        y_true_f = K.flatten(y_true_class)
        y_pred_f = K.flatten(y_pred_class)
        
        intersection = K.sum(y_true_f * y_pred_f)
        union = K.sum(y_true_f) + K.sum(y_pred_f)
        
        dice = (2. * intersection + smooth) / (union + smooth)
        dice_sum += dice
    return dice_sum

def combined_loss(y_true, y_pred,):
    #dice loss
    dice_total = dice_sum(y_true, y_pred)
    dice_mean = dice_total / 4.0
    dice_loss =  1.0 - dice_mean

    #focal loss
    gamma=2.0
    alpha=0.25
    epsilon = K.epsilon()

    y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

    ce = -y_true * K.log(y_pred)
    focal_weight = alpha * K.pow(1 - y_pred, gamma)
    focal_loss = focal_weight * ce

    focal_loss = K.sum(focal_loss, axis=-1)
    focal_loss = K.mean(focal_loss)
    
    return 0.5 * dice_loss + 0.5 * focal_loss


In [ ]:
#metrics
def dice_coefficient_multiclass(y_true, y_pred):
    return dice_sum(y_true, y_pred) / 4.0

def dice_coefficient_per_class(class_idx, class_name):
    def dice(y_true, y_pred, smooth=1e-6):
        y_true_class = y_true[..., class_idx]
        y_pred_class = y_pred[..., class_idx]
        
        y_true_f = K.flatten(y_true_class)
        y_pred_f = K.flatten(y_pred_class)
        
        intersection = K.sum(y_true_f * y_pred_f)
        union = K.sum(y_true_f) + K.sum(y_pred_f)
        
        return (2. * intersection + smooth) / (union + smooth)
    
    dice.__name__ = f'dice_{class_name}'
    return dice

In [ ]:
#compile model
from keras.optimizers import Adam
mean_iou = tf.keras.metrics.MeanIoU(num_classes=CLASSES, name='mean_iou')

model.compile(optimizer=Adam(1e-4),
                loss=combined_loss,
                metrics=[dice_coefficient_multiclass,
                        dice_coefficient_per_class(0, "background"),
                        dice_coefficient_per_class(1, "glioma"),
                        dice_coefficient_per_class(2, "meningioma"),
                        dice_coefficient_per_class(3, "pituitary"),
                        mean_iou
                        ]
            )

In [8]:
#setup gpu
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [ ]:
# training
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard
train_generator = SegmentationGenerator(
    train_images_arr, train_masks_arr,
    batch_size=4,
    classes=4,
    augmentation=training_augmentation,
    shuffle=True
)

val_gen = SegmentationGenerator(
    val_images_arr, val_masks_arr,
    batch_size=4,
    classes=4,
    augmentation=validation_augmentation,
    shuffle=False
)

callbacks = [
    ModelCheckpoint('tumor_segmentation_best.keras', monitor='val_loss',verbose=1, save_best_only=True, mode='min'   ),
    EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=8, verbose=1, min_lr=1e-6),
    CSVLogger('training_tumor_segmentation.csv'),
    TensorBoard(log_dir='./logs/tensorboard')
]

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks,
    use_multiprocessing=True,
    workers= WORKERS,
    verbose=1
)

model.save('tumor_segmentation_epoch100.keras')